In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install chromadb

In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb


class EmbeddingFunction:
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    def __call__(self, input):
        return self.model.encode(input).tolist()


class DB:
    def __init__(self, distance_function, root_path):
        self.ef = EmbeddingFunction()
        self.client = chromadb.PersistentClient(path=root_path)
        self.distance_function = distance_function
        assert distance_function in ["l2", "ip", "cosine"], "Distance function should be 'l2' or 'ip' or 'cosine'"
        self.collection = self.client.get_or_create_collection("lab5_" + self.distance_function,
                                                               metadata={"hnsw:space": self.distance_function},
                                                               embedding_function=self.ef)

    def add(self, items):
        old_batch = 0
        new_batch = 1000
        while True:
            if new_batch > len(items["fragments"]):
                break
            self.collection.add(
                documents=items["fragments"][old_batch:new_batch],
                metadatas=items["metadata"][old_batch:new_batch],
                ids=items["ids"][old_batch:new_batch])
            old_batch = new_batch
            new_batch += 1000
        self.collection.add(
            documents=items["fragments"][old_batch:],
            metadatas=items["metadata"][old_batch:],
            ids=items["ids"][old_batch:])

    def query(self, query, n_results):
        return self.collection.query(query_embeddings=self.ef(query), n_results=n_results)

    def clear(self):
        self.client.delete_collection("lab5_" + self.distance_function)
        self.collection = self.client.get_or_create_collection("lab5_" + self.distance_function,
                                                               metadata={"hnsw:space": self.distance_function},
                                                               embedding_function=self.ef)

In [ ]:
database_l2 = DB("l2", "/content/drive/MyDrive/ITMO/NLP/lab5/DB")

In [ ]:
!pip install ctransformers

In [ ]:
!pip install tiktoken
!pip install openai
!pip install cohere
!pip install kaleido

In [ ]:
!pip install gradio==3.48.0

In [ ]:
from ctransformers import AutoModelForCausalLM
import gradio as gr

In [ ]:
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q2_K.gguf", model_type="mistral", gpu_layers=50)


In [ ]:
def process_request(message, history):
    context = "\n".join(database_l2.query(message, 5)["documents"][0])
    prompt = "Answer the question using provided context. Your answer should be in your own words and be no longer than 50 words.\n"
    prompt += "Context: " + context + "\n"
    prompt += "Question: " + message + "\n"
    prompt += "Answer: "
    answer = llm(prompt)
    return f"{prompt}\n\n{answer}"

1. Проблемы фондового рынка в разгар летнего спада?
2. Что говорится в рекомендации по безопасности?
3. Кому проиграл Трит Хьюи?
4. Что вызвало гнев азиатских соседей?
5. Где жили НАТО когда прибыли в Боснию?
6. Как звали первых клонированных кошек?
7. Кто такой Шумахер?
8. Кто такой Путин?
9. Почему Владимир Путин молодец?
0. Почему Трамп победил на выборах?
11. Как вернуться в 2007?
12. Раньше было лучше?

1. 655 Что улучшит состояние пациентов находящихся на диализе? 655
What will improve the condition of patients on dialysis?
2. Как изменились зарплаты топ менеджеров?
How have the salaries of top managers changed?
3. 709 How do they work to improve their image? Как работают над улучшением имиджа?
4. 839 На что влияют рейды ополченцев? What is the impact of militiamen raids?
5. 1006 Какой ущерб от урагана Чарли во флориде?
How much damage did Hurricane Charlie cause in Florida?
6. 1047 Какие инциденты происходили на атомной электростанции? What incidents occurred at a nuclear power plant?
7. 1150 Что сейчас с тренером Los Angeles Galaxy?
What's up with the Los Angeles Galaxy coach now?
8. 1240 Какую партию США называют террористической? Which US party is called terrorist?
9. 1287 In which players renew their vows as teammates? Какие игроки возобновляют свои клятвы товарищей по команде?
10. 1337 Что происходит в Багдаде? What's happening in Baghdad?


In [ ]:
demo = gr.ChatInterface(fn=process_request, title="Chat bot")
demo.launch()